In [2]:
import pandas as pd
from sklearn.base import TransformerMixin
import re

### Inserting dataset

In [3]:
data = pd.read_csv('data/Real411_election_data/real_411_election_complaints_2021_11_30.csv')

In [4]:
data.head(5)

,id,source_url,title,overview,outcome,status,platform,asset_urls,asset_text_extracts,created_at
0,1813,https://twitter.com/PartyOfAction,Twitter: Breach of ECC?,The complaint is about a Twitter account that ...,22 October 2021 10:12:59|Logged by: Rampershad...,Resolved,Twitter,https://s3-eu-west-1.amazonaws.com/complaints-...,PARTY OF ACTION - Political Party IEC Register...,2021-09-17 08:21:03
1,1815,https://twitter.com/PartyOfAction,NaN,NaN,NaN,Resolved,Twitter,https://s3-eu-west-1.amazonaws.com/complaints-...,NaN,2021-09-17 11:21:14
2,1821,NaN,NaN,NaN,NaN,Needs More Information,WhatsApp,https://s3-eu-west-1.amazonaws.com/complaints-...,a\r\n21% I 6:52\r\nNaeem Patel\r\n5\r\nminutes...,2021-09-22 06:21:53
3,1824,360sheq@gmail.com,Possible Voter Registration Fraud,Person complained that they were registered in...,Not Relevant to Real411. Please contact IEC r...,Resolved - Out of Scope,Other,https://s3-eu-west-1.amazonaws.com/complaints-...,-27820070059\r\nUse OTP 517392 to proceed with...,2021-09-22 14:20:55
4,1826,https://twitter.com/mark__rountree/status/1441...,Helen Zille posts Peter de Villiers' announcem...,The video shared by Helen Zille on her Twitter...,Update and clarification: We have been advised...,Resolved,Facebook,https://s3-eu-west-1.amazonaws.com/complaints-...,NaN,2021-09-24 10:32:18


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   42 non-null     int64 
 1   source_url           34 non-null     object
 2   title                34 non-null     object
 3   overview             34 non-null     object
 4   outcome              34 non-null     object
 5   status               42 non-null     object
 6   platform             42 non-null     object
 7   asset_urls           36 non-null     object
 8   asset_text_extracts  33 non-null     object
 9   created_at           42 non-null     object
dtypes: int64(1), object(9)
memory usage: 3.4+ KB


In [6]:
data["asset_text_extracts"][0]

'PARTY OF ACTION - Political Party IEC Registere\r\n4,327 Tweets\r\nIF YOU DO NOT TAKE AN INTEREST IN THE\r\nAFFAIRS OF YOUR GOVERNMENT. THEN YOU ARE\r\nDOOMED TO LIVE UNDER THE RULE OF FOOLS.\r\nPLATO\r\n"9 -\r\nPOA\r\n...\r\nFollow\r\nPARTY OF ACTION - Political Party IEC Registered\r\n)PartyOfAction\r\n#Naturallmmunity #COVIDSurvivor #SAvaxrecord #NoLockDowns\r\n#FreedomOfReligion #NoMedicalApartheid #NoMandatoryVaccines\r\n#NoMandatoryMasks\r\no\r\nSouth Africa\r\nIII\r\nJoined July 2020\r\n1,424 Following 2,105 Followers\r\nn\r\nFollowed by Karabo Mokgoko\r\nand Peter Ndoro | '

In [7]:
df = data[data["asset_text_extracts"].notna()]

In [8]:
df

,id,source_url,title,overview,outcome,status,platform,asset_urls,asset_text_extracts,created_at
0,1813,https://twitter.com/PartyOfAction,Twitter: Breach of ECC?,The complaint is about a Twitter account that ...,22 October 2021 10:12:59|Logged by: Rampershad...,Resolved,Twitter,https://s3-eu-west-1.amazonaws.com/complaints-...,PARTY OF ACTION - Political Party IEC Register...,2021-09-17 08:21:03
2,1821,NaN,NaN,NaN,NaN,Needs More Information,WhatsApp,https://s3-eu-west-1.amazonaws.com/complaints-...,a\r\n21% I 6:52\r\nNaeem Patel\r\n5\r\nminutes...,2021-09-22 06:21:53
3,1824,360sheq@gmail.com,Possible Voter Registration Fraud,Person complained that they were registered in...,Not Relevant to Real411. Please contact IEC r...,Resolved - Out of Scope,Other,https://s3-eu-west-1.amazonaws.com/complaints-...,-27820070059\r\nUse OTP 517392 to proceed with...,2021-09-22 14:20:55
5,1827,www.na.co.za,NaN,NaN,NaN,Needs More Information,Other,https://s3-eu-west-1.amazonaws.com/complaints-...,|,2021-09-25 21:10:02
6,1828,https://twitter.com/Guerrillaz2/status/1441770...,Twitter: Color determines the level of corrupt...,The complaint is based on a poster by the EFF ...,The poster is clear misinformation because it ...,Resolved,Twitter,https://s3-eu-west-1.amazonaws.com/complaints-...,IT IS NOT\r\nCORRUPTION IF\r\nIT IS BLACK\r\n#...,2021-09-25 22:09:54
7,1834,https://www.facebook.com/1209371845/posts/1022...,Facebook: Fake ballot papers misleading voters,Complaint is based on a candidate who could po...,The nature of the complaint is unclear. The sa...,Resolved,Facebook,https://s3-eu-west-1.amazonaws.com/complaints-...,SAMPLE BALLOT PAPER\r\nDISTRICT COUNCIL PARTY ...,2021-09-30 09:11:57
9,1841,NaN,Whatsapp: Special vote,The complaint is based on an infographic publi...,\r\nThe inforgraphic is slightly misleading as...,Resolved,WhatsApp,https://s3-eu-west-1.amazonaws.com/complaints-...,DA\r\nSkip the lines\r\nVote early!\r\nIf you ...,2021-10-02 06:36:21
12,1851,https://twitter.com/Sakhumuzi__/status/1445384...,Twitter: Offensive DA posters,The complaint is about offensive posters that ...,22 October 2021 10:03:58|Logged by: Rampershad...,Resolved,Twitter,https://s3-eu-west-1.amazonaws.com/complaints-...,THE\r\nANC\r\nCALLED\r\nYOU\r\nRACISTS\r\nTHE\...,2021-10-05 15:44:58
14,1855,Www.IEC.co.za,NaN,NaN,NaN,Not Relevant,Other,https://s3-eu-west-1.amazonaws.com/complaints-...,12:03\r\n3ul\r\nVo))\r\n4G\r\nS\r\nLTE\r\nSign...,2021-10-08 10:47:34
15,1858,www.steenbokpan.co.za,NaN,NaN,NaN,Needs More Information,Other,https://s3-eu-west-1.amazonaws.com/complaints-...,IVERNMENTA\r\nETIONS 2021\r\nDE\r\nANC\r\n2021 |,2021-10-08 17:58:23


In [9]:
# # class for cleaning the text
# class CleanTextTransformer(TransformerMixin):
#     def transform(self, X, **transform_params):
#         return [cleanText(text) for text in X]
#     def fit(self, X, y=None, **fit_params):
#         return self
#     def get_params(self, deep=True):
#             return {}

def cleanText(text):
    text = [elem.strip().replace("\n", " ").replace("\r", " ") for elem in text]
    return text

In [10]:
df["asset_text_extracts"] = cleanText(list(df["asset_text_extracts"]))

<ipython-input-10-caae17944713>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["asset_text_extracts"] = cleanText(list(df["asset_text_extracts"]))


In [12]:
df["asset_text_extracts"]

0     PARTY OF ACTION - Political Party IEC Register...
2     a  21% I 6:52  Naeem Patel  5  minutes.ag9go  ...
3     -27820070059  Use OTP 517392 to proceed with  ...
5                                                     |
6     IT IS NOT  CORRUPTION IF  IT IS BLACK  #REVOLU...
7     SAMPLE BALLOT PAPER  DISTRICT COUNCIL PARTY VO...
9     DA  Skip the lines  Vote early!  If you are a ...
12    THE  ANC  CALLED  YOU  RACISTS  THE  DA  CALLS...
14    12:03  3ul  Vo))  4G  S  LTE  Sign in to Stand...
15             IVERNMENTA  ETIONS 2021  DE  ANC  2021 |
16    11:25  81160061006393  Monday, Sep 20 . 15:45 ...
17                                                    |
20    Herman Mashaba a  @HermanMashaba  As @Action4S...
21    Herman Mashaba  @HermanMashaba  I am sorry fel...
22                                   alon  102  1/307 |
23    Dear Ratepayers  My aim is to continue to enga...
24    Wesley Fester  @wesleyfestersa  Malema: I got ...
26    11:55  all 63%  K  Tweet  Stop the ANC-IRA

In [13]:
def remove_url(txt):
    """Replace URLs found in a text string with nothing 
    (i.e. it will remove the URL from the string).

    Parameters
    ----------
    txt : string
        A text string that you want to parse and remove urls.

    Returns
    -------
    The same txt string with url's removed.
    """

    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

In [15]:
# Remove URLs
df["asset_text_extracts"]= df["asset_text_extracts"].apply(remove_url)

<ipython-input-15-1b1f2e3b6d10>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["asset_text_extracts"]= df["asset_text_extracts"].apply(remove_url)


In [16]:
df["asset_text_extracts"]

0     PARTY OF ACTION Political Party IEC Registere ...
2     a 21 I 652 Naeem Patel 5 minutesag9go Happy Ga...
3     27820070059 Use OTP 517392 to proceed with you...
5                                                      
6     IT IS NOT CORRUPTION IF IT IS BLACK REVOLUTIONARY
7     SAMPLE BALLOT PAPER DISTRICT COUNCIL PARTY VOT...
9     DA Skip the lines Vote early If you are a regi...
12    THE ANC CALLED YOU RACISTS THE DA CALLS YOU HE...
14    1203 3ul Vo 4G S LTE Sign in to Standard Bank ...
15                   IVERNMENTA ETIONS 2021 DE ANC 2021
16    1125 81160061006393 Monday Sep 20 1545 The hol...
17                                                     
20    Herman Mashaba a HermanMashaba As Action4SA we...
21    Herman Mashaba HermanMashaba I am sorry fellow...
22                                        alon 102 1307
23    Dear Ratepayers My aim is to continue to engag...
24    Wesley Fester wesleyfestersa Malema I got vacc...
26    1155 all 63 K Tweet Stop the ANCIRASAPA co

In [17]:
del df['id']
del df['source_url']
del df['asset_urls']

In [18]:
del df['outcome']

In [19]:
del df['created_at']

In [20]:
df

,title,overview,status,platform,asset_text_extracts
0,Twitter: Breach of ECC?,The complaint is about a Twitter account that ...,Resolved,Twitter,PARTY OF ACTION Political Party IEC Registere ...
2,NaN,NaN,Needs More Information,WhatsApp,a 21 I 652 Naeem Patel 5 minutesag9go Happy Ga...
3,Possible Voter Registration Fraud,Person complained that they were registered in...,Resolved - Out of Scope,Other,27820070059 Use OTP 517392 to proceed with you...
5,NaN,NaN,Needs More Information,Other,
6,Twitter: Color determines the level of corrupt...,The complaint is based on a poster by the EFF ...,Resolved,Twitter,IT IS NOT CORRUPTION IF IT IS BLACK REVOLUTIONARY
7,Facebook: Fake ballot papers misleading voters,Complaint is based on a candidate who could po...,Resolved,Facebook,SAMPLE BALLOT PAPER DISTRICT COUNCIL PARTY VOT...
9,Whatsapp: Special vote,The complaint is based on an infographic publi...,Resolved,WhatsApp,DA Skip the lines Vote early If you are a regi...
12,Twitter: Offensive DA posters,The complaint is about offensive posters that ...,Resolved,Twitter,THE ANC CALLED YOU RACISTS THE DA CALLS YOU HE...
14,NaN,NaN,Not Relevant,Other,1203 3ul Vo 4G S LTE Sign in to Standard Bank ...
15,NaN,NaN,Needs More Information,Other,IVERNMENTA ETIONS 2021 DE ANC 2021


In [21]:
df.to_csv("real411-election-cleaned_data.csv")